In [ ]:
import data.download_yf_data as yf_download
from data.pull_data import pull_yfinance_sample_data 
from mom_trans.data_prep import deep_momentum_strategy_features

%load_ext autoreload
%autoreload 2

In [ ]:
# yf_download.main(index_symbol="NDX")

In [ ]:
deep_momentum_strategy_features(pull_yfinance_sample_data("AAPL"))